# Test Structured Output / Structured response

Some models and APIs have support for structured output, even defining and enforcing a schema and types for the generated output. Is that usable enough for us to use it? Which models support it?

## Testing with AI Sandbox

This is the [OpenAI post](https://openai.com/index/introducing-structured-outputs-in-the-api/) I started with.

Then I found Azure OpenAI documentation on [structured outputs](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure%2Cdotnet-entra-id&pivots=programming-language-python).

Documentation includes the list of supported models; this includes version dates, but I'm not sure we have access to that information for the AI Sandbox models.

It _also_ lists the API version, which might be why I couldn't get this to work:

> Support for structured outputs was first added in API version `2024-08-01-preview`. It is available in the latest preview APIs as well as the latest GA API: `2024-10-21`.

As of April 2025, the API version to use with AI Sandbox is **2025-03-01-preview**.

* * *

Use a page of text from the quote data subset for testing.

This is the last entry in the quote data subset file. The expected quote based on annotation data:

> „die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“

In [ ]:
# define some variables with sample content and prompts to use in a few different calls

page_text = """
Das demokrattsche Prinzip und seine Anwendung. 25 liegt das auf der Hand. Wir mögen bis an den Eingang zur Werkstatt gleich¬ sein, aber in der Werkstatt sind wir es nicht mehr. Da muß der Ingenieur anordnett und der Schlosser, Dreher &c. ausführen, da kann der Heizer nicht nach seinem Kopf verfahren und den Kessel abstellen, wenn es ihm paßt. So¬ in jedem großen Wirthschaftsunternehmen, so aber auch in der Wirthschaft selbst. Ueberall, wo Kooporation ist, ist Arbeitstheilung, und wo Arbeitstheilung ist, ist Verschiedenheit der Funktionen, wo Verschiedenheit der Funktionen Verschieden¬ heit der Vollmachten. Diese sind heute vielfach übertrieben, weil das überkommene Nr C Klassenmoment hineinspielt, weil der Hert Ingenieur in der Regel der he¬ schenden Gesellschaftsklasse angehört und der Dreher der beherrschten. Diese Ueber¬ treibung, der Absolutismus in der Werkstatt &c., läßt sich beseitigen und wird im Lautfe der Entwicklung beseitigt werden. Aber eben nur die Uebertreibung, die Differenzirung wird darum doch bleiben. Sie wird nur ihre Schärfe dadurch¬ verlieren, daß die Menschen selbst vielseitiger ausgebildet und vielseitiger beschäftigt, werden, so daß die Unterordnung wechselt. Die Sozialdemokratie kann sich nicht außerhalb der Gesellschaft stellen, der sie lebt, kann also auch in ihren Reihen die thatsächlichen Unterschiede nicht ignoriren. Es wird immer ihr Bestreben sein müssen, für jeden Posten den möglichst geeigneten Mann herauszusuchen, und das trifft auch für die Vertretung im Parlament 3u. In Uebrigen sind für die Verwirklichung der Demokratie noch wichtigere, Aufgaben zu erfüllen als die Verbesserung der Stimmenzählungsmethoden. Sehr viel wichtiger ist die Demokratisirung der Verwaltungen, die bessere Vertheilung der Verwaltungsaufgaben und die Demokratisirung des Wahlrechts zu den Ver¬ waltungskörpern. Ob die Arbetterklasse statt durch 45 durch 95 Abgeordnete, im Reichstag vertreten ist, das würde an den Dingen vorderhand wenig ändern, denn die Gesetze würden kaum viel anders ausfallen als jetzt. Aber noch ist der Eintritt in die meisten Landtage, in die Provinzial- und Kreisvertretungen. den Arbeitern verschlossen, und in den Gemeindevertretungen nur mit großen Einschränkungen möglich. Das möchte Manchem hente als gleichgiltig erscheinen gegenüber den großen Erfolgen bei den Reichstagswahlen. Ohue diese zu ver¬ kleinern müssen wir jedoch daran erinnern, daß diese Erfolge zum Theil das Produkt außergewöhnlicher Umstände sind, und daß im Uebrigen „die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“ Wir erkennen also an, daß innerhalb gewisser Greuzen und unter bestimmten. erhältnissen — sehr vorgeschrittene politische Einrichtungen - has Proportional¬ wahlsystem wünschbar sein mag. In Deutschland, wo noch so viele Elementar¬ bedingungen demokratischen Lebens fehlen, ist es ein Luxusartikel, für den Kraft¬ einzusetzen sie wichtigeren Arbeiten entziehen hieße. — Beispiele dafür giebt es schon heute. So kommen bei sogenannten freiwilligen Feuerwehren Subordinationsverhältnisse vor, die den bürgerlichen Lebensstellungen der be¬ treffenden Personen direkt widersprechen. Desgleichen beim Heer, und sie würden dort noch hüufiger sein, wenn nicht in Deutschland bei den Heereseinrichtungen dem ständischen Prinzip Rechnung getragen würde. NEW_DOCUMENT
Zinner,+etal._1896_15:01_388_Notizen,Feuilleton
"""

system_prompt = "You are a helpful research assistant fluent in German. You help researchers identify important content in text from German scholarship.  Identify and return any passages in this text provided by the user that quote from works by Karl Marx."

user_prompt = page_text

In [ ]:
import json
import os
from typing import Optional

import marimo as mo
import openai
from openai import AzureOpenAI
from pydantic import BaseModel


# define a simple model with the fields we want returned
# previously tried including start/end indices; model returns numbers but they are useless
class Quote(BaseModel):
    text: str
    title: Optional[str]


# Both APIs supports nested models; we need to support multiple quotes on a page, so return a list of quotes
# using the Quote model defined above
class QuoteList(BaseModel):
    quotes: list[Quote]


# initialize an api client for AI sandbox

SANDBOX_ENDPOINT = "https://api-ai-sandbox.princeton.edu/"
SANDBOX_API_VERSION = "2025-03-01-preview"

client = AzureOpenAI(
    api_key=os.getenv("AI_SANDBOX_KEY"),
    api_version=SANDBOX_API_VERSION,
    azure_endpoint=SANDBOX_ENDPOINT,
)

### gpt-4o

For structured response with gpt-4o, we use openai + pydantic to pass in the response model to the `tools` option.

If you try passing it in as a response format (`response_format=Quote`), you get a BadRequest response with this error message:
```
response_format value as json_schema is enabled only for api versions 2024-08-01-preview and later
```

In [ ]:
gpt4o_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    # model="gpt-4o-mini",
    # model="Meta-Llama-3-1-8B-Instruct-nwxcg",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    # response_format=QuoteList.model_json_schema(),  # not supported by our API version or not being passed correctly
    tools=[
        openai.pydantic_function_tool(QuoteList),
    ],
)

Dump the full response as json:

In [ ]:
# here is the full response as json
print(gpt4o_completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-BPCDd1oRG1MPxzSYvV0KGrvpG9REI",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_I2CzFKfaWNb3dfpZtpJSF6C6",
            "function": {
              "arguments": "{\"quotes\": [{\"text\": \"„die Arbeiterklasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“\", \"title\": null}]}",
              "name": "QuoteList",
              "parsed_arguments": {
                "quotes": [
                  {
                    "text": "„die Arbeiterklasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“",
                    "title": null
                  }
         

Output the parsed response:

In [ ]:
# this parsed_arguments field is actually a Quote instance!
gpt4o_parsed = (
    gpt4o_completion.choices[0].message.tool_calls[0].function.parsed_arguments
)

mo.md(f"""
Returned {len(gpt4o_parsed.quotes)} quote(s).

**quotation:**

{gpt4o_parsed.quotes[0].text}


**title:**
{gpt4o_parsed.quotes[0].title}


""")

Returned 1 quote(s). 
 quotation: 
 „die Arbeiterklasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“ 
 title: 
None

**GPT4o** : ✅ quote  ⛔title

Correctly returns the full text of the expected quote and only that quote.

It looks like it is using the article title (?) at the top of the page text as the title.  The tag in the annotation data is "Manifest der Kommunistischen Partei"

### gpt-4o-mini

In [ ]:
gpt4omini_completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    # model="Meta-Llama-3-1-8B-Instruct-nwxcg",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    # response_format=Quote,  # not supported by our API version
    tools=[
        openai.pydantic_function_tool(QuoteList),
    ],
)

In [ ]:
print(gpt4omini_completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-BPCDf2QRPDsNQAsRmzFFrmfbraLDX",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_WJsG9MMKVOOIAslovGwbKidc",
            "function": {
              "arguments": "{\"quotes\":[{\"text\":\"die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann\",\"title\":null}]}",
              "name": "QuoteList",
              "parsed_arguments": {
                "quotes": [
                  {
                    "text": "die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann",
                    "title": null
                  }
             

In [ ]:
# this parsed_arguments field is actually a Quote instance!
gpt4omini_parsed = (
    gpt4omini_completion.choices[0]
    .message.tool_calls[0]
    .function.parsed_arguments
)

mo.md(f"""
Returned {len(gpt4omini_parsed.quotes)} quote(s).

**quotation:**

{gpt4omini_parsed.quotes[0].text}


**title:**
{gpt4omini_parsed.quotes[0].title}
""")

Returned 1 quote(s). 
 quotation: 
 die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann 
 title: 
None

**GPT4o-mini** : ⛔/✅quote  ⛔title

On a previous run it returned incorrect quote: a different set of text surrounded by „“.  This time it returned the correct quote.

Like GPT4o, it returns the text at the beginning of the page as a title.

### ⛔ llama 3.1 8B instruct

I couldn't get this one to work. When I try the `tools` parameters that works for gpt4o and our version of the API:
```python
tools=[
        openai.pydantic_function_tool(Quote),
    ],
```
I get an "invalid input error." The details of the message indicate it's complaining about required fields (maybe required fields for the quote object).

When I try specifying it as a response format, I get different errors. Passing in the class:
```python
response_format=Quote
```
Results in
> Response format was json_schema but must be either 'text' or 'json_object'.

When I try passing the json schema for my model, I get the same error:
```python
response_format=Quote.model_json_schema()
```

In [ ]:
llama_completion = client.beta.chat.completions.parse(
    model="Meta-Llama-3-1-8B-Instruct-nwxcg",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    # not sure the right syntax for this one
    response_format=Quote.model_json_schema(),
)

## Testing with local Ollama server

Testing with the Ollama python client based on a [blog post about structured outputs](https://ollama.com/blog/structured-outputs).

Setup requires installing [ollama](https://ollama.com/), and then start the server and download (pull) and run models.

```console
pip install ollama
ollama serve
ollama run llama3.2
ollama run mixtral
```

You can use `ollama ps` to check which models are running and how much longer they will be running; default keepalive time is 5 minutes.

In [ ]:
from ollama import chat


def identify_quotes(page_text, model):
    response = chat(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        model=model,
        format=QuoteList.model_json_schema(),
    )

    return QuoteList.model_validate_json(response.message.content)

### llama3.2

In [ ]:
# try getting from llama3.2
llama_quotes = identify_quotes(page_text, "llama3.2")

In [ ]:
llama_quotes_md = [f"Identified {len(llama_quotes.quotes)} quote(s)"]

for llama_q in llama_quotes.quotes:
    llama_quotes_md.append(f"""**quotation:**    
{llama_q.text}

**title:**
{llama_q.title}""")

mo.md("\n\n\n".join(llama_quotes_md))

Identified 1 quote(s) 
 quotation: 
Klassenmoment 
 title: 
Quelle: Marx, Karl (1867): Das Kapital. Kritik der politischen Ökonomie. Erster Teil. Herausgegeben von Friedrich Engels. Stuttgart: J.H.W. Dietz. S. 91.

**llama3.2** : ⛔/0️⃣ quote  ⛔/0️⃣title

On some runs it returned an incorrect quote and incorrect title. It was returning some text from the third line of the first paragraph. When I adjusted the prompt to move the instructions to the system prompt, it didn't return anything.

### mixtral

In [ ]:
mixtral_quotes = identify_quotes(page_text, "mixtral")

In [ ]:
mixtral_quotes_md = [f"Identified {len(mixtral_quotes.quotes)} quote(s)"]

for mixtral_q in mixtral_quotes.quotes:
    mixtral_quotes_md.append(f"""**quotation:**    
{mixtral_q.text}

**title:**
{mixtral_q.title}""")

mo.md("\n\n\n".join(mixtral_quotes_md))

Identified 1 quote(s) 
 quotation: 
Wir erkennen also an, daß innerhalb gewisser Grenzen und unter bestimmten, erhaltnissen — sehr vorgeschrittene politische Einrichtungen - has Proportional-wahlsystem wünschbar sein mag. 
 title: 
Zinner, etal. 1896 15:01 388 Notizen, Feuilleton

**mixtral** : ⛔quote  ⛔title

Returning multiple things, all of them wrong. On a previous run, before I modified the prompt, it returned some text near the beginning of the first paragraph (also incorrect).

Ollama server is [compatible with openai chat completions API](https://ollama.com/blog/openai-compatibility), including structured responses.

In [ ]:
from openai import OpenAI


ollama_oaiclient = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # required, but unused
)

In [ ]:
ollama_completion = ollama_oaiclient.beta.chat.completions.parse(
    model="llama3.2",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    # response_format=QuoteList.model_json_schema(),  # this seems to be ignored
    tools=[
        openai.pydantic_function_tool(QuoteList),
    ],
)

<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/Users/rkoeser/workarea/env/marx/lib/python3.13/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">141</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
<span class="w">    </span><span class="pm">~~~~^^^^^^^^^^^^^^^^^^</span>
  File <span class="nb">&quot;/var/folders/mb/6qm4h4yx3yqdy2bv2sjyp4z00000gp/T/marimo_22610/__marimo__cell_aqbW_.py&quot;</span>, line <span class="m">1</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">ollama_completion</span> <span class="o">=</span> <span class="n">ollama_oaiclient</span><span class="o">.</span><sp

In [ ]:
print(ollama_completion.model_dump_json(indent=2))

The API is not exactly compatible, since the `tool_calls` is null. The content looks like a json dump of the QuoteList object with associated quotes, so it is enforcing the structure.

## CSV output

CSV output might be a simpler way to get structured output, but may be less reliable.

Testing with the last paragraph of page text from the quote subset (page index 661, two quotes).

The expected quote from this paragraph:
> „Zur Lösung dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er versprach, diese Lösung später zu geben.

### ollama - llama3.2

In [ ]:
csv_prompt = """Identify any direct quotes of Karl Marx's works in this paragraph, along with the work title if known. Also include the source for the title (context reference or otherwise). Return the results in this CSV format:
number,quote,title,title_source

Nachdem Marx im ersten Bande des „Kapital“, Kapitel X, das Gesetz, des Mehrwerths festgestellt hatte, fügte er sogleich hinzu, „daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht". „Zur Lösung dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er versprach, diese Lösung später zu geben. Die wenigen Nationalökonomen, denen diese Stelle auffiel, setzten ihre Zuversicht auf diesen ihnen unlösbar scheinenden Widerspruch, der nach ihrer Ansicht die Theorie zu Falle bringen mußte. Mehrere hofften, daß der Theore¬ tiker des Werthes dort mit seiner Dialektik und seinem Kommumnismus scheitern. werde, denen, wie sie überzeugt waren, jede wissenschaftliche Grundlage fehle. Herr Loria, der geniale Entdecker so mancher schon von Marx entdeckten Theorien, ging so weit, zu behaupten, daß derselbe, um seine Ohumacht nicht einzugestehen, sich entschlossen hätte, die beiden Bände, welche sein ökonomisches
"""

csvresponse = chat(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": csv_prompt},
    ],
    model="llama3.2",
)
csvresponse

ChatResponse(model='llama3.2', created_at='2025-04-22T18:01:42.871681Z', done=True, done_reason='stop', total_duration=2507974667, load_duration=11408583, prompt_eval_count=396, prompt_eval_duration=217995875, eval_count=227, eval_duration=2277371250, message=Message(role='assistant', content='Unfortunately, I couldn\'t find any direct quotes of Karl Marx\'s works in the provided paragraph. However, I can suggest some possible sources where you might find relevant information.\n\nThat being said, I did manage to extract a reference to a Marxist concept:\n\n1.,,"daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht",,,Marx, Kapital (Erstes Buch), S. 123\n(Note: This is not a direct quote from Marx\'s work, but rather an paraphrase of his ideas)\n\nIf you would like me to analyze the paragraph further or try to find relevant quotes, please let me know!\n\nEdit:\n\nAfter re-reading the paragraph, I found another possible reference:\n\n1.,"Zur Lösung dieses Widerspruchs",,,Marx, Kapital (Erstes Buch), S. 123\nAgain, this is not a direct quote from Marx\'s work, but rather an attribution of his ideas.\n\nIf you have any further information or context about the paragraph, I may be able to help you better.', images=None, tool_calls=None))

In [ ]:
print(csvresponse.message.content)

Unfortunately, I couldn't find any direct quotes of Karl Marx's works in the provided paragraph. However, I can suggest some possible sources where you might find relevant information.

That being said, I did manage to extract a reference to a Marxist concept:

1.,,"daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht",,,Marx, Kapital (Erstes Buch), S. 123
(Note: This is not a direct quote from Marx's work, but rather an paraphrase of his ideas)

If you would like me to analyze the paragraph further or try to find relevant quotes, please let me know!

Edit:

After re-reading the paragraph, I found another possible reference:

1.,"Zur Lösung dieses Widerspruchs",,,Marx, Kapital (Erstes Buch), S. 123
Again, this is not a direct quote from Marx's work, but rather an attribution of his ideas.

If you have any further information or context about the paragraph, I may be able to help you better.


### AI sandbox - gpt-4o

In [ ]:
# try ai sandbox gpt-4o with csv output

gpt4o_csv_completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": csv_prompt},
    ],
)

In [ ]:
print(gpt4o_csv_completion.choices[0].message.content)

number,quote,title,title_source
1,"daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht","Kapital","Kapital, Kapitel X, Band 1"
2,"Zur Lösung dieses Widerspruchs","Kapital","Kapital, Kapitel X, Band 1"


The second quote found is the expected one, but the full content is not returned.

### AI sandbox - gpt-4o-min

In [ ]:
gpt4omini_csv_completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": csv_prompt},
    ],
)

In [ ]:
print(gpt4omini_csv_completion.choices[0].message.content)

number,quote,title,title_source
1,"daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht","Das Kapital, Band I","Kapital, Erste Auflage"
2,"Zur Lösung dieses Widerspruchs","Das Kapital, Band I","Kapital, Erste Auflage"


The second quote is the expected response. It doesn't return the full text, but returns more of the quote than GPT4o did. They seem to be stopping at punctuation - likely because this quote as annotated includes text _between_ and _after_ the parts of the quote.

> „Zur Lösung dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er versprach, diese Lösung später zu geben.

From google translate:

> "To resolve this contradiction," he continues, "many intermediate links are still needed." He promised to provide this solution later.